In [1]:
# this notebook is for plotting with the ATLAS "work in progress" or "simulation internal" labels

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from hydra.utils import instantiate
from hydra import initialize, compose
import hydra
import wandb

from data.dataManager import DataManager
from model.modelCreator import ModelCreator
from omegaconf import OmegaConf
from scripts.run import setup_model, load_model_instance

/home/dkraft/.local/lib/python3.10/site-packages/wandb/apis/public.py:3105: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
[21:21:35.841] INFO   CaloQuVAE                                         Loading configuration.
2025-08-27 21:21:36,498 dwave.cloud INFO  MainThread Log level for 'dwave.cloud' namespace set to 0
[21:21:36.498] INFO   dwave.cloud                                       Log level for 'dwave.cloud' namespace set to 0


In [2]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path="config")
config=compose(config_name="config.yaml")
wandb.init(tags = [config.data.dataset_name], project=config.wandb.project, entity=config.wandb.entity, config=OmegaConf.to_container(config, resolve=True), mode='disabled')

/home/dkraft/.local/lib/python3.10/site-packages/wandb/apis/public.py:3105: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [3]:
# check which dev:
devids = ["cuda:{0}".format(x) for x in list(config.gpu_list)]
dev = torch.device(devids[0])
print(devids)

['cuda:3']


In [10]:
# loading in the model
new_model = False
if new_model:
    self = setup_model(config)
    # self.model = self.model.double()  # sets all model parameters to float64
else:
    #self = load_model_instance(config.config_path)
    self = load_model_instance(config)
    # self.model = self.model.double()

[21:24:20.447] INFO   data.dataManager                                  Loading other dataset: CaloChallenge2
[21:24:20.449] INFO   data.dataManager                                  Keys: ['incident_energies', 'showers']
[21:24:26.334] INFO   data.dataManager                                  dict_keys(['incident_energies', 'showers'])
[21:24:26.335] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f9fd4379c60>: 79999 events, 157 batches
[21:24:26.336] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f9fd4379de0>: 10001 events, 10 batches
[21:24:26.336] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f9fd4379cf0>: 9999 events, 10 batches
[21:24:26.337] INFO   model.modelCreator                                ::Creating Model
[21:24:27.066] WARN   model.rbm.zephyr                                  QPU is off

/home/dkraft/CaloQuVAE


[21:24:27.345] INFO   scripts.run                                       Requesting GPUs. GPU list :[3]
[21:24:27.347] INFO   scripts.run                                       Main GPU : cuda:3
[21:24:27.347] INFO   scripts.run                                       CUDA available
[21:24:27.461] INFO   model.modelCreator                                Loading state


cuda:3
encoder._networks.0.seq1.0.conv.weight True
encoder._networks.0.seq1.0.conv.bias True
encoder._networks.0.seq1.1.weight True
encoder._networks.0.seq1.1.bias True
encoder._networks.0.seq1.2.weight True
encoder._networks.0.seq1.3.conv.weight True
encoder._networks.0.seq1.3.conv.bias True
encoder._networks.0.seq1.4.weight True
encoder._networks.0.seq1.4.bias True
encoder._networks.0.seq1.5.weight True
encoder._networks.0.seq1.6.conv.weight True
encoder._networks.0.seq1.6.conv.bias True
encoder._networks.0.seq1.7.weight True
encoder._networks.0.seq1.7.bias True
encoder._networks.0.seq1.8.weight True
encoder._networks.0.seq2.0.conv.weight True
encoder._networks.0.seq2.0.conv.bias True
encoder._networks.0.seq2.1.weight True
encoder._networks.0.seq2.1.bias True
encoder._networks.0.seq2.2.weight True
encoder._networks.0.seq2.3.conv.weight True
encoder._networks.0.seq2.3.conv.bias True
encoder._networks.0.seq2.4.weight True
encoder._networks.1.seq1.0.conv.weight True
encoder._networks.1.

[21:24:27.903] INFO   model.modelCreator                                Loading weights from file : /fast_scratch_1/caloqvae/jtoledo/wandb/run-20250721_152921-c22o2vxp/files/ae_separate_249.pth


Loading weights for module =  _hit_smoothing_dist_mod
Loading weights for module =  _bce_loss
Loading weights for module =  encoder
Loading weights for module =  decoder
Loading weights for module =  prior


In [13]:
# uncomment this to run evaluate vae and show plots:
self.evaluate_vae(self.data_mgr.val_loader, epoch=0)

[21:25:22.862] INFO   engine.engine                                     Epoch: 0 - Average Val Loss: 15010.9894


{'val_ae_loss': 13458.0109375,
 'val_hit_loss': 1565.1702026367188,
 'val_entropy': -121.91909790039062,
 'val_loss': 15010.98935546875}

In [18]:
# function to plot with ATLAS labels:
def plot_histograms(ax, target, recon, sampled, xlabel, ylabel, title, bins=30, log_scale=True):
    max_value = max(target.max(), recon.max(), sampled.max())
    min_value = min(target.min(), recon.min(), sampled.min())
    if min_value == max_value:
        print("Warning: min and max values are the same, adjusting to avoid division by zero.")
        max_value += 0.1
    binning = np.arange(min_value, max_value, (max_value - min_value) / bins)

    ax.hist(target, histtype="stepfilled", bins=binning, density=True, alpha=0.7, label='Ground Truth', color='b', linewidth=2.5)
    ax.hist(recon, histtype="step", bins=binning, density=True, label='Reconstruction', color='c', linewidth=2.5)
    ax.hist(sampled, histtype="step", bins=binning, density=True, label='Sampled', color='orange', linewidth=2.5)

    ax.set_xlabel(xlabel, fontsize=16)
    ax.set_ylabel(ylabel, fontsize=16)
    ax.set_title(title, fontsize=18)
    ax.set_yscale('log' if log_scale else 'linear')
    ax.set_ylim(top=ax.get_ylim()[1] * 1.7)
    ax.grid(True)
    #ax.legend(fontsize=15, loc='lower center')
    ax.legend(fontsize=15, loc='lower left')

    # ATLAS label
    top = ax.get_ylim()[1]
    left = ax.get_xlim()[0]
    right = ax.get_xlim()[1]
    height = top - ax.get_ylim()[0]
    width = right - left
    
    ax.text(0.05, 0.93, 'ATLAS', fontsize=19,
        fontweight='bold', style='italic',
        transform=ax.transAxes, horizontalalignment='left')

    ax.text(0.22, 0.93, 'Work in Progress', fontsize=18,
        transform=ax.transAxes, horizontalalignment='left')

    fig = ax.get_figure()
    fig.tight_layout()

In [19]:
# layer specific deposited energy:

# adding ATLAS label to the plots:
# cfg = self._config
# layer_cell_count = cfg.data.r * cfg.data.phi
# relevant_layers = cfg.data.relevantLayers

# for i, layer_num in enumerate(relevant_layers):
#     idx_prev = i * layer_cell_count
#     idx = (i + 1) * layer_cell_count

#     target_layer = self.showers[:, idx_prev:idx]
#     recon_layer = self.showers_recon[:, idx_prev:idx]
#     sampled_layer = self.showers_prior[:, idx_prev:idx]

#     target_energy = torch.sum(target_layer, dim=1).numpy() / 1000  # convert to GeV
#     recon_energy = torch.sum(recon_layer, dim=1).numpy() / 1000
#     sampled_energy = torch.sum(sampled_layer, dim=1).numpy() / 1000

#     fig = plt.figure(figsize=(7, 6))
#     ax = plt.gca()
#     plot_histograms(ax, target_energy, recon_energy, sampled_energy,
#                     xlabel='Deposited Energy (GeV)', ylabel='Density',
#                     title=f'Layer {layer_num} - Deposited Energy')
#     fig.savefig(f'PosterPlots/layer{layer_num}_deposited_energy.png', dpi=300)
#     plt.closefig


In [20]:
# energy conditioned sparsity:

# incident_np = self.incident_energy.view(-1).numpy()
# target_sparsity = ((self.showers == 0).sum(dim=1) / self.showers.shape[1]).numpy()
# recon_sparsity = ((self.showers_recon == 0).sum(dim=1) / self.showers_recon.shape[1]).numpy()
# sampled_sparsity = ((self.showers_prior == 0).sum(dim=1) / self.showers_prior.shape[1]).numpy()

# # energy bin centers for ATLAS
# energy_bin_centers = [2 ** i for i in range(8, 23)] 

# for i, energy_center in enumerate(energy_bin_centers):
#     e_low = 2 ** (np.log2(energy_center) - 0.5)
#     e_high = 2 ** (np.log2(energy_center) + 0.5)

#     mask = (incident_np >= e_low) & (incident_np < e_high)
#     if mask.sum() == 0:
#         continue

#     fig = plt.figure(figsize=(7, 6))
#     ax = plt.gca()
#     plot_histograms(
#         ax,
#         target_sparsity[mask],
#         recon_sparsity[mask],
#         sampled_sparsity[mask],
#         xlabel='Sparsity',
#         ylabel='Density',
#         title=f'Sparsity ~ {e_low / 1000:.1f} - {e_high / 1000:.1f} GeV'
#     )
#     fig.savefig(f'PosterPlots/sparsity_energybin_{i}.png', dpi=300)
#     plt.close(fig)


In [21]:
# layer specific incident energy:

# layer_cell_count = cfg.data.r * cfg.data.phi
# relevant_layers = cfg.data.relevantLayers
# incident = self.incident_energy.view(-1).numpy()
# epsilon = 1e-7  # avoid division errors

# for i, layer_num in enumerate(relevant_layers):
#     idx_prev = i * layer_cell_count
#     idx = (i + 1) * layer_cell_count

#     target_layer = self.showers[:, idx_prev:idx]
#     recon_layer = self.showers_recon[:, idx_prev:idx]
#     sampled_layer = self.showers_prior[:, idx_prev:idx]

#     target_energy = torch.sum(target_layer, dim=1).numpy()
#     recon_energy = torch.sum(recon_layer, dim=1).numpy()
#     sampled_energy = torch.sum(sampled_layer, dim=1).numpy()

#     target_ratio = target_energy / (incident + epsilon)
#     recon_ratio = recon_energy / (incident + epsilon)
#     sampled_ratio = sampled_energy / (incident + epsilon)

#     fig = plt.figure(figsize=(7, 6))
#     ax = plt.gca()
#     plot_histograms(
#         ax,
#         target_ratio,
#         recon_ratio,
#         sampled_ratio,
#         xlabel='Deposited / Incident Energy',
#         ylabel='Density',
#         title=f'Layer {layer_num} - Incidence Ratio'
#     )
#     fig.savefig(f'PosterPlots/layer{layer_num}_incidence_ratio.png', dpi=300)
#     #plt.close(fig)


In [22]:
# adding in RBM plotting with the ATLAS label

def plot_rbm_histogram(rbm_post, rbm_prior, rbm_prior_qpu=None, save_path=None):
    energy_post = rbm_post.numpy()
    energy_prior = rbm_prior.numpy()

    if rbm_prior_qpu is not None:
        energy_qpu = rbm_prior_qpu.numpy()
        minVal = min(energy_post.min(), energy_prior.min(), energy_qpu.min())
        maxVal = max(energy_post.max(), energy_prior.max(), energy_qpu.max())
    else:
        minVal = min(energy_post.min(), energy_prior.min())
        maxVal = max(energy_post.max(), energy_prior.max())

    binwidth = (maxVal - minVal) / 30
    bins = np.arange(minVal, maxVal + binwidth, binwidth)

    fig = plt.figure(figsize=(8, 6))
    ax = plt.gca()

    ax.hist(energy_post, bins=bins, linewidth=2.5, color="b", density=True,
            log=True, label="RBM Posterior", alpha=0.7)
    
    ax.hist(energy_prior, bins=bins, color="orange", density=True,
            histtype='step', linewidth=2.5, label="RBM Prior")

    if rbm_prior_qpu is not None:
        ax.hist(energy_qpu, bins=bins, color="m", density=True,
                histtype='step', linewidth=2.5, label="RBM QPU")

    ax.set_xlabel("RBM Energy", fontsize=16)
    ax.set_ylabel("Density", fontsize=16)
    ax.set_title("RBM Energy Distribution", fontsize=16)
    ax.set_yscale("log")
    ax.grid(True)
    ax.legend(fontsize=14, loc='lower center')
    ax.set_ylim(top=ax.get_ylim()[1] * 1.8)
    
    # adding ATLAS style label
    ax.text(0.05, 0.93, 'ATLAS', fontsize=19, fontweight='bold', style='italic',
            transform=ax.transAxes, horizontalalignment='left')
    ax.text(0.20, 0.93, 'Work in Progress', fontsize=18,
            transform=ax.transAxes, horizontalalignment='left')
    
    fig.tight_layout()

    if save_path:
        fig.savefig(save_path, dpi=300)
        #plt.close(fig)
    
    return fig


In [23]:
# calling function to plot:
#fig_rbm = plot_rbm_histogram(self.RBM_energy_post, self.RBM_energy_prior,
#                             save_path="PosterPlots/rbm_energy_histogram.png")
